Eliminate Implication

In [2]:
def eliminate_implication(expression):
    while expression.find('=>') != -1:
        subS = expression.split("=>",1)
        i = len(subS[0])-1
        while i != 0:
            if subS[0][i] == '(':
                newS = subS[0][:i-1] + ' ~' + subS[0][i-1:].strip()
                expression = f"{newS} | {subS[1].strip()}"
                break
            i = i - 1
    return expression



In [3]:
print(eliminate_implication("(x | (y => z))"))
print(eliminate_implication("(q | (p => (q => r)))"))

(x | ~(y | z))
(q | ~(p | ~(q | r)))


Move Negation Inward

In [4]:
def Negation(expression):
    negation = False
    Fexp = ""
    indoor = False
    for char in expression:

        if char == "~" and indoor:
            Fexp += char

        elif char == "~" and not negation:
            negation = True

        elif char == "|":
            Fexp += "&" if negation else "|"

        elif char == "&":
            Fexp += "|" if negation else "&"

        elif char == "(" and negation:
            indoor = True
            Fexp += char

        elif char == "(" and not negation:
            indoor = True
            Fexp += char

        elif char == ")" and negation:
            Fexp += char
            negation = False
            indoor = False

        elif char == " ":
            Fexp += char

        else:
            Fexp += "~" if negation else ""
            Fexp += char
    return Fexp


def move_negation_inward(expression):
    i=0
    Fexp = ''
    while i != len(expression):
        if expression[i] == '~' and expression[i+1] == '(':
            Fexp = expression[:i] + Negation(expression[i:])
        i+=1
    return Fexp



In [5]:
print(move_negation_inward("~(~q & ~p)"))
print(move_negation_inward("(q | ~(p | (q | r)))"))
# (q | (~p & ~(~q & ~w)))

(~~q | ~~p)
(q | (~p & (~q & ~r)))


Remove Double Not

In [6]:
def RemovedoubleNot(expression):
    Fexp = ""
    cnt = 0
    for char in expression:

        if char == "~":
            cnt += 1
       
        else:
            if cnt % 2 != 0 and cnt != 0:
                Fexp += "~"
            Fexp += char
            cnt = 0

    return Fexp


In [7]:
print(RemovedoubleNot("(~q | ~~(p | ~~~(q | ~w)))"))
print(RemovedoubleNot("((~~T(w) | Q(w)) | (~~~Q(x) | ~T(x)))"))

(~q | (p | ~(q | ~w)))
((T(w) | Q(w)) | (~Q(x) | ~T(x)))


Standardize Variables

In [8]:
def Standardizevariables(expression):
    Fexp = ""
    sympols = []
    ch = ""
    operations = ["|", "&", "="]
    replace = False

    for char in expression:

        if char in sympols and replace:
            ch = chr(ord(sympols[-1]) + 1)
            Fexp += ch

        else:
            if ord(char) >= 97 and ord(char) <= 122:
                sympols.append(char)
                replace = False

            else:
                if char in operations:
                    replace = True
                
                elif char == ")":
                    replace = False
                    if ch != "":
                        sympols.append(ch)
            Fexp += char
    return Fexp


In [9]:
print(Standardizevariables("(∀x P(x)) | (∃x Q(x)) & (∀z Q(z))"))
print(Standardizevariables("(∀x P(x)) | (∃y Q(y)) & (∀x Q(x))"))

(∀x P(x)) | (∃y Q(y)) & (∀z Q(z))
(∀x P(x)) | (∃y Q(y)) & (∀z Q(z))


The Prenex Form

In [10]:
def ThePrenexForm(expression):
    Fexp = ""
    subS = " "

    for char in expression:
        
        if char == "∀" or char == "∃":
            subS += char

        elif subS[-1] == "∃" or subS[-1] == "∀":
            subS += char + " "
        elif char ==' ' and Fexp[-1] == '(':
            continue
        else:
            Fexp += char

    Fexp = subS.strip()+ ' ' + Fexp
    return Fexp


In [11]:
print(ThePrenexForm("(∀x P(x)) | (∃x Q(x)) & (∀z Q(z))"))
print(ThePrenexForm('(∀y (∃z (∀w (P(w) ^ Q(y) ^ T(z))))'))

∀x ∃x ∀z (P(x)) | (Q(x)) & (Q(z))
∀y ∃z ∀w ((((P(w) ^ Q(y) ^ T(z))))


Skolemization


In [12]:
def Skolemize(expression):
    
    replaced = []
    universal = False
    Fexp = ''
    sign = ''
    subS= ''

    for char in expression:
        if char == '∀':
            universal = True
            sign = char
            Fexp += char

        elif sign == '∀':
            sign = char
            Fexp += char
        
        elif char == "∃":
            subS += char

        elif len(subS) != 0 and subS[-1] == "∃":
            subS += char
            replaced.append(char)

        else:
            if char in replaced:
                if universal:
                    Fexp += f'F({sign})'
                else:
                    Fexp += 'B' if 'B' not in replaced else chr(ord(replaced[-1]) + 1)
                    replaced.append(Fexp[-1])
            else:
                Fexp += char
    return Fexp.strip()

In [13]:
print(Skolemize("∃y ∃z P(A) | Q(y) & T(z)"))
print(Skolemize("∀x ∃y P(A) | Q(y) & T(y)"))

P(A) | Q(B) & T(C)
∀x  P(A) | Q(F(x)) & T(F(x))


Eliminate Univarsel Quantifiers

In [14]:
def Eliminate_universal_quantifiers(expression):
    Fexp = ''
    subS = ''
    for char in expression:
        if char == "∀" :
            subS += char

        elif subS[-1] == "∀":
            subS += char 

        else:
            if char == ' ' and len(Fexp) == 0: continue
            Fexp += char
    
    return Fexp

In [15]:
print(Eliminate_universal_quantifiers("∀x ∀x ∀z ( P(x) | Q(x) & Q(z) )"))
print(Eliminate_universal_quantifiers("∀y ∀z ∀w  ( P(w) ^ Q(y) ^ T(z) )"))

( P(x) | Q(x) & Q(z) )
( P(w) ^ Q(y) ^ T(z) )
